In [21]:
import pandas as pd

csv_file_path = "Data/ebay_comments.csv"
df = pd.read_csv(csv_file_path)
df




,Search,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,musicmagpie,£634.99,NaN,Probably as I had hoped for as the phone is in...,Great but heavy.,4 stars,stickyprobs,"28 Jun, 2023",Yes
1,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,ssa2234ssa,"Fantastic,new and sealed as described. It come...",Fantastic,5 stars,helwa1078-voxze...,"29 Sep, 2022",Yes
2,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,pre-owned,£634.99,phones2gadgets,It’s a good phone.. Had a new 13 model but fo...,Will do for me for a few years.. hopefully..,5 stars,paulprior249,"20 Jun, 2023",Yes
3,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,phonesdirect_com,£634.99,NaN,Perfect quality as if its new and it has 100% ...,Goood,5 stars,88s275,"16 Aug, 2023",Yes
4,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,sell-phones,I love it fantastic phone,Perfect 👍,5 stars,sarolga,"04 Jun, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...,...
62642,Apple Watch,"Bidding ended on Thu, 23 Nov at 11:00 AM.",pre-owned,£195.00,davdivjak,Amazing very pleased with it had no problems w...,Good buy,5 stars,bigtez1,"11 Aug, 2023",Yes
62643,Apple Watch,"Bidding ended on Thu, 23 Nov at 11:00 AM.",new,£195.00,marstan57,You know what you’re getting with an apple pro...,Fantastic watch,5 stars,kevicarte-7,"08 Jun, 2023",Yes
62644,Apple Watch,"Bidding ended on Thu, 23 Nov at 11:00 AM.",new,£195.00,gogipat,Amazing,Gry,5 stars,stan3530,"12 Sep, 2023",Yes
62645,Apple Watch,"Bidding ended on Thu, 23 Nov at 11:00 AM.",new,£195.00,shelly1719,Yes,Very good,5 stars,semifav0,"12 May, 2023",Yes


In [22]:
results = df.to_dict(orient='records')
len(results)

62647

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

iphone_url = "https://www.ebay.co.uk/sch/i.html?_from=R40&_trksid=p4432023.m570.l2632&_nkw=apple+watch&_sacat=178893"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
urls = list()

for i in range(167):
    print(i)
    try:
        search_url = iphone_url + str(i+1)

        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        temp = [a["href"] for a in soup.find_all("a", class_="s-item__link")]

        urls = urls + temp
    except:
        break
    time.sleep(2)

    # Filter out non-product links
urls = [link for link in urls if "ebay.co.uk/itm/" in link]
urls = list(set(urls))
urls


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [ ]:
import pandas as pd

# Initialize an empty list to store the results
#results = []

for url in range(len(urls)):
    print(url)
    # Send a GET request to the URL
    response = requests.get(urls[url], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements
    try:
        product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    except:
        continue
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    
    # Get the text from the elements
    try:
        product_title = product_title_element.get_text(strip=True)
    except:
        continue
    price = price_element.find(class_="ux-textspans").text.strip()
    try:
        reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    except:
        continue
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    pages_list = list()
    
    for i in range(pages_count):
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page, headers=headers)
        
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements
        review_elements = soup.find_all(class_="ebay-review-section")
        

        # Loop through the review elements and extract the text
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
                
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            results.append({'Search': 'Apple Watch', 'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})
            df = pd.DataFrame(results)
            df.to_csv('ebay_comments.csv', index=False)
        time.sleep(2)
        
df


In [ ]:
#df = pd.DataFrame(results)
df = df.drop_duplicates()
df.to_csv('ebay_comments.csv', index=False)
df

In [ ]:
df.Search.value_counts()

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

iphone_url = "https://www.ebay.co.uk/sch/58058/i.html?_from=R40&_nkw=macbook&LH_TitleDesc=0"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
urls = list()

for i in range(167):
    print(i)
    try:
        search_url = iphone_url + str(i+1)

        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        temp = [a["href"] for a in soup.find_all("a", class_="s-item__link")]

        urls = urls + temp
    except:
        break
    time.sleep(2)

    # Filter out non-product links
urls = [link for link in urls if "ebay.co.uk/itm/" in link]
urls = list(set(urls))
urls


In [ ]:
import pandas as pd

# Initialize an empty list to store the results
#results = []

for url in range(len(urls)):
    print(url)
    # Send a GET request to the URL
    response = requests.get(urls[url], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements
    try:
        product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    except:
        continue
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    
    # Get the text from the elements
    try:
        product_title = product_title_element.get_text(strip=True)
    except:
        continue
    price = price_element.find(class_="ux-textspans").text.strip()
    try:
        reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    except:
        continue
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    pages_list = list()
    
    for i in range(pages_count):
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page, headers=headers)
        
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements
        review_elements = soup.find_all(class_="ebay-review-section")
        

        # Loop through the review elements and extract the text
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
                
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            results.append({'Search': 'Macbook', 'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})
            df = pd.DataFrame(results)
            df.to_csv('ebay_comments.csv', index=False)
        time.sleep(2)
        
df


In [ ]:
#df = pd.DataFrame(results)
df = df.drop_duplicates()
df.to_csv('ebay_comments.csv', index=False)
df

In [ ]:
df.Search.value_counts()

In [12]:
df.Search.value_counts()

Iphone         51903
AirPods         5231
iPad            2354
Apple Watch     1828
Macbook         1331
Name: Search, dtype: int64

In [ ]:
Mac
iPhone
iPad
AirPods
Watch
